MAIN PACKAGES AND LIBRAIRIES


In [ ]:

# To install

!pip install kagglehub


# To import 

import kagglehub
import pandas            as pd
import numpy             as np
import matplotlib.pyplot as plt
import seaborn           as sns

from datetime                 import datetime
from sklearn.model_selection  import train_test_split
from sklearn.preprocessing    import LabelEncoder, StandardScaler
from sklearn.metrics          import classification_report, roc_auc_score, confusion_matrix, roc_curve, auc 
from colorama                 import Fore, Style, Back, init


# Download latest version
path = kagglehub.dataset_download("kartik2112/fraud-detection")
print("Path to dataset files:", path)



# Legend


"""
index - Unique Identifier for each row
trans_date_trans_time - Transaction DateTime
cc_num - Credit Card Number of Customer
merchant - Merchant Name
category - Category of Merchant
amt - Amount of Transaction
first - First Name of Credit Card Holder
last - Last Name of Credit Card Holder
gender - Gender of Credit Card Holder
street - Street Address of Credit Card Holder
city - City of Credit Card Holder
state - State of Credit Card Holder
zip - Zip of Credit Card Holder
lat - Latitude Location of Credit Card Holder
long - Longitude Location of Credit Card Holder
city_pop - Credit Card Holder's City Population
job - Job of Credit Card Holder
dob - Date of Birth of Credit Card Holder
trans_num - Transaction Number
unix_time - UNIX Time of transaction
merch_lat - Latitude Location of Merchant
merch_long - Longitude Location of Merchant
is_fraud - Fraud Flag <--- Target Class
"""


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\fabio\anaconda3\lib\site-packages\pip\_internal\cli\base_command.py", line 106, in _run_wrapper
    status = _inner_run()
  File "C:\Users\fabio\anaconda3\lib\site-packages\pip\_internal\cli\base_command.py", line 97, in _inner_run
    return self.run(options, args)
  File "C:\Users\fabio\anaconda3\lib\site-packages\pip\_internal\cli\req_command.py", line 67, in wrapper
    return func(self, options, args)
  File "C:\Users\fabio\anaconda3\lib\site-packages\pip\_internal\commands\install.py", line 484, in run
    installed_versions[distribution.canonical_name] = distribution.version
  File "C:\Users\fabio\anaconda3\lib\site-packages\pip\_internal\metadata\pkg_resources.py", line 192, in version
    return parse_version(self._dist.version)
  File "C:\Users\fabio\anaconda3\lib\site-packages\pip\_v

Path to dataset files: C:\Users\fabio\.cache\kagglehub\datasets\kartik2112\fraud-detection\versions\1


"\nindex - Unique Identifier for each row\ntrans_date_trans_time - Transaction DateTime\ncc_num - Credit Card Number of Customer\nmerchant - Merchant Name\ncategory - Category of Merchant\namt - Amount of Transaction\nfirst - First Name of Credit Card Holder\nlast - Last Name of Credit Card Holder\ngender - Gender of Credit Card Holder\nstreet - Street Address of Credit Card Holder\ncity - City of Credit Card Holder\nstate - State of Credit Card Holder\nzip - Zip of Credit Card Holder\nlat - Latitude Location of Credit Card Holder\nlong - Longitude Location of Credit Card Holder\ncity_pop - Credit Card Holder's City Population\njob - Job of Credit Card Holder\ndob - Date of Birth of Credit Card Holder\ntrans_num - Transaction Number\nunix_time - UNIX Time of transaction\nmerch_lat - Latitude Location of Merchant\nmerch_long - Longitude Location of Merchant\nis_fraud - Fraud Flag <--- Target Class\n"

STEP 1 - ANALYSIS

In [40]:
# Load the dataset

df_train = pd.read_csv(f"{path}\\fraudTrain.csv")
df_test = pd.read_csv(f"{path}\\fraudTest.csv")

# First analysis

from colorama import Fore, Style, Back

text       = "First values of the train model"
color      = Fore.LIGHTMAGENTA_EX + Style.BRIGHT
color_text = Fore.LIGHTCYAN_EX + Style.BRIGHT
line       = "~" * (len(text) + 10)

print(color + line)
print(color_text + f"~~~  {text}  ~~~")
print(color + line + Style.RESET_ALL)

df_train.head(3)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~  First values of the train model  ~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0


We can observe a lot of irrevelant datas in this dataset. Then, we decide to clean and use only the useful variables.

In [55]:
df_train["trans_date_trans_time"] = pd.to_datetime(df_train["trans_date_trans_time"], errors="coerce")

df_train["hour"]  = df_train["trans_date_trans_time"].dt.hour
df_train["day"]   = df_train["trans_date_trans_time"].dt.day
df_train["month"] = df_train["trans_date_trans_time"].dt.month

drop_columns = [
    "Unnamed: 0",
    "first",
    "last",
    "street",
    "city",
    "state",
    "zip",
    "dob",
    "job",
    "trans_num"
]

text       = "Trans date time converted in different columns"
color      = Fore.LIGHTMAGENTA_EX + Style.BRIGHT
color_text = Fore.LIGHTCYAN_EX + Style.BRIGHT
line       = "~" * (len(text) + 10)

print(color + line)
print(color_text + f"~~~  {text}  ~~~")
print(color + line + Style.RESET_ALL)

df_train.head(3)

df_train = df_train.drop(columns=[c for c in drop_columns if c in df_train.columns])
df_train.head(3)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~  Trans date time converted in different columns  ~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


,trans_date_trans_time,cc_num,merchant,category,amt,gender,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud,hour,day,month,distance
0,2019-01-01 00:00:18,2703186189652095,514,8,4.97,0,36.0788,-81.1781,3495,1325376018,36.011293,-82.048315,0,0,1,1,78.238288
1,2019-01-01 00:00:44,630423337322,241,4,107.23,0,48.8878,-118.2105,149,1325376044,49.159047,-118.186462,0,0,1,1,1.752746
2,2019-01-01 00:00:51,38859492057661,390,0,220.11,1,42.1808,-112.2620,4154,1325376051,43.150704,-112.154481,0,0,1,1,8.790583


In [54]:
cat_col  = ["merchant", "category", "gender"]
encoders = {}

for col in cat_col:
    encoders[col] = LabelEncoder()
    df_train[col] = encoders[col].fit_transform(df_train[col])

def haversine(lat_customer, long_customer, lat_merch, long_merch):
    R = 6371
    lat_customer, long_customer, lat_merch, long_merch = map(np.radians, [lat_customer, long_customer, lat_merch, long_merch])

    dist_lat  = lat_merch  - lat_merch
    dist_long = long_merch - long_customer

    a = np.sin(dist_lat/2)**2 + np.cos(lat_customer) * np.cos(lat_merch) * np.sin(dist_long/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    
    distance = R * c
    return distance

text       = "Distance column added in dataset"
color      = Fore.LIGHTMAGENTA_EX + Style.BRIGHT
color_text = Fore.LIGHTCYAN_EX + Style.BRIGHT
line       = "~" * (len(text) + 10)

print(color + line)
print(color_text + f"~~~  {text}  ~~~")
print(color + line + Style.RESET_ALL)

df_train.head(3)

df_train["distance"] = haversine(df_train["lat"], df_train["long"], df_train["merch_lat"], df_train["merch_long"])
df_train.head(10)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~  Distance column added in dataset  ~~~
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


,trans_date_trans_time,cc_num,merchant,category,amt,gender,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud,hour,day,month,distance
0,2019-01-01 00:00:18,2703186189652095,514,8,4.97,0,36.0788,-81.1781,3495,1325376018,36.011293,-82.048315,0,0,1,1,78.238288
1,2019-01-01 00:00:44,630423337322,241,4,107.23,0,48.8878,-118.2105,149,1325376044,49.159047,-118.186462,0,0,1,1,1.752746
2,2019-01-01 00:00:51,38859492057661,390,0,220.11,1,42.1808,-112.2620,4154,1325376051,43.150704,-112.154481,0,0,1,1,8.790583
3,2019-01-01 00:01:16,3534093764340240,360,2,45.00,1,46.2306,-112.1138,1939,1325376076,47.034331,-112.561071,0,0,1,1,34.149485
4,2019-01-01 00:03:06,375534208663984,297,9,41.96,1,38.4207,-79.4629,99,1325376186,38.674999,-78.632459,0,0,1,1,72.218116
5,2019-01-01 00:04:08,4767265376804500,607,2,94.63,0,40.3750,-75.2045,2158,1325376248,40.653382,-76.152667,0,0,1,1,80.152878
6,2019-01-01 00:04:42,30074693890476,534,3,44.54,0,37.9931,-100.9893,2691,1325376282,37.162705,-100.153370,0,0,1,1,73.662798
7,2019-01-01 00:05:08,6011360759745864,107,2,71.65,1,38.8432,-78.6003,6018,1325376308,38.948089,-78.540296,0,0,1,1,5.192862
8,2019-01-01 00:05:18,4922710831011201,250,9,4.27,0,40.3359,-79.6607,1472,1325376318,40.351813,-79.958146,0,0,1,1,25.208469
9,2019-01-01 00:06:01,2720830304681674,563,4,198.39,0,36.5220,-87.3490,151785,1325376361,37.179198,-87.485381,0,0,1,1,12.134652


In [27]:
df_train = pd.read_csv(f"{path}\\fraudTrain.csv")
df_test = pd.read_csv(f"{path}\\fraudTest.csv")

# First analysis

print("test columns : \n \n",df_test.columns)
print("\n \n train columns : \n \n",df_train.columns)

# Remove unnecessary index
if 'Unnamed: 0' in df_train.columns:
    df_train = df_train.drop('Unnamed: 0', axis=1)
    df_test = df_test.drop('Unnamed: 0', axis=1)

print(f"\n TRAIN: {df_train.shape[0]:,} rows × {df_train.shape[1]} columns")
print(f" TEST: {df_test.shape[0]:,} rows × {df_test.shape[1]} columns")
print(f"\n Columns:\n{list(df_train.columns)}")

df = df_train.copy()

# ============================================================================
# 2. DATA QUALITY
# ============================================================================

print("\n" + "="*80)
print("2. DATA QUALITY")
print("="*80)

# Missing values
missing_train = df_train.isnull().sum().sum()
missing_test = df_test.isnull().sum().sum()
print(f"\n Missing values - TRAIN: {missing_train}, TEST: {missing_test}")

# Duplicates
dup_train = df_train.duplicated().sum()
dup_test = df_test.duplicated().sum()
print(f" Duplicates - TRAIN: {dup_train}, TEST: {dup_test}")

# Data types
print(f"\n Data types:")
print(df.dtypes.value_counts())

# ============================================================================
# 3. CLASS IMBALANCE (CRITICAL)
# ============================================================================

print("\n" + "="*80)
print("3. CLASS IMBALANCE - TARGET 'is_fraud'")
print("="*80)

target_col = 'is_fraud'

# TRAIN distribution
class_dist_train = df_train[target_col].value_counts().sort_index()
class_pct_train = (df_train[target_col].value_counts(normalize=True).sort_index() * 100)
imbalance_ratio_train = class_dist_train[0] / class_dist_train[1]

print(f"\n TRAIN:")
print(f"   Legitimate (0): {class_dist_train[0]:,} ({class_pct_train[0]:.3f}%)")
print(f"   Fraud (1): {class_dist_train[1]:,} ({class_pct_train[1]:.3f}%)")
print(f"   Imbalance ratio: {imbalance_ratio_train:.0f}:1")

# TEST distribution
class_dist_test = df_test[target_col].value_counts().sort_index()
class_pct_test = (df_test[target_col].value_counts(normalize=True).sort_index() * 100)
imbalance_ratio_test = class_dist_test[0] / class_dist_test[1]

print(f"\n TEST:")
print(f"   Legitimate (0): {class_dist_test[0]:,} ({class_pct_test[0]:.3f}%)")
print(f"   Fraud (1): {class_dist_test[1]:,} ({class_pct_test[1]:.3f}%)")
print(f"   Imbalance ratio: {imbalance_ratio_test:.0f}:1")

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].bar([0, 1], class_dist_train.values, color=['#2ecc71', '#e74c3c'], 
           edgecolor='black', linewidth=1.5)
axes[0].set_title('TRAIN Distribution', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Class', fontsize=11)
axes[0].set_ylabel('Count', fontsize=11)
axes[0].set_xticks([0, 1])
axes[0].set_xticklabels(['Legitimate', 'Fraud'])
for i, v in enumerate(class_dist_train.values):
    axes[0].text(i, v, f'{v:,}\n({class_pct_train.values[i]:.3f}%)', 
               ha='center', va='bottom', fontweight='bold')

axes[1].bar([0, 1], class_dist_test.values, color=['#3498db', '#e67e22'], 
           edgecolor='black', linewidth=1.5)
axes[1].set_title('TEST Distribution', fontsize=13, fontweight='bold')
axes[1].set_xlabel('Class', fontsize=11)
axes[1].set_ylabel('Count', fontsize=11)
axes[1].set_xticks([0, 1])
axes[1].set_xticklabels(['Legitimate', 'Fraud'])
for i, v in enumerate(class_dist_test.values):
    axes[1].text(i, v, f'{v:,}\n({class_pct_test.values[i]:.3f}%)', 
               ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.savefig('class_imbalance.png', dpi=300, bbox_inches='tight')
plt.show()

# ============================================================================
# 4. KEY FEATURES ANALYSIS
# ============================================================================

print("\n" + "="*80)
print("4. KEY FEATURES")
print("="*80)

# Identify feature types
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

if target_col in numeric_cols:
    numeric_cols.remove(target_col)

print(f"\n Feature types:")
print(f"   Numeric: {len(numeric_cols)}")
print(f"   Categorical: {len(categorical_cols)}")

# Amount analysis
print(f"\n Amount (amt) statistics:")
print(f"   Mean: ${df['amt'].mean():.2f}")
print(f"   Median: ${df['amt'].median():.2f}")
print(f"   Std: ${df['amt'].std():.2f}")
print(f"   Range: ${df['amt'].min():.2f} - ${df['amt'].max():.2f}")

print(f"\n   By class:")
for cls in [0, 1]:
    label = "Legitimate" if cls == 0 else "Fraud"
    mean_amt = df[df[target_col] == cls]['amt'].mean()
    print(f"   {label}: ${mean_amt:.2f}")

# Temporal feature
df['trans_datetime'] = pd.to_datetime(df['trans_date_trans_time'])
print(f"\n Time period:")
print(f"   From: {df['trans_datetime'].min()}")
print(f"   To: {df['trans_datetime'].max()}")
print(f"   Duration: {(df['trans_datetime'].max() - df['trans_datetime'].min()).days} days")


# ============================================================================
# 5. BASIC STATISTICS
# ============================================================================

print("\n" + "="*80)
print("5. DESCRIPTIVE STATISTICS")
print("="*80)

print("\n Numeric features summary:")
print(df[numeric_cols].describe().T[['mean', 'std', 'min', 'max']])

# ============================================================================
# 6. KEY INSIGHTS & NEXT STEPS
# ============================================================================

print("\n" + "="*80)
print("6. KEY INSIGHTS & NEXT STEPS")
print("="*80)

print("\n KEY FINDINGS:")
print(f"   1. Severe class imbalance: ~{imbalance_ratio_train:.0f}:1 ratio")
print(f"   2. {len(numeric_cols)} numeric features, {len(categorical_cols)} categorical")
print(f"   3. High cardinality in merchant, job → Need encoding strategy")
print(f"   4. Temporal data available for feature engineering")
print(f"   5. No missing values (excellent data quality)")

print("\n PREPROCESSING ROADMAP:")
print("   1. Drop PII: cc_num, first, last, street, trans_num")
print("   2. Temporal features: extract hour, day, day_of_week from trans_date_trans_time")
print("   3. Encode categoricals:")
print("      • Low cardinality (gender, state): One-Hot or Label Encoding")
print("      • High cardinality (category, merchant, job): Target Encoding")
print("   4. Scale numeric features: amt, lat, long, city_pop")
print("   5. Feature engineering: customer-merchant distance, age from dob")
print("   6. Handle imbalance: SMOTE, class_weight, or undersampling")

print("\n MODELING STRATEGY:")
print("   • Validation: StratifiedKFold (5-fold)")
print("   • Metrics: F1-Score (primary), PR-AUC, ROC-AUC")
print("   • Models: Random Forest, Gradient Boosting, XGBoost")
print("   • Optimize threshold based on business cost (FP vs FN)")

print("\n" + "="*80)
print(" EXPLORATION COMPLETED - Ready for preprocessing!")
print("="*80)

test columns : 
 
 Index(['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category',
       'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip',
       'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time',
       'merch_lat', 'merch_long', 'is_fraud'],
      dtype='object')

 
 train columns : 
 
 Index(['Unnamed: 0', 'trans_date_trans_time', 'cc_num', 'merchant', 'category',
       'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip',
       'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time',
       'merch_lat', 'merch_long', 'is_fraud'],
      dtype='object')

 TRAIN: 1,296,675 rows × 22 columns
 TEST: 555,719 rows × 22 columns

 Columns:
['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time', 'merch_lat', 'merch_long', 'is_fraud']

2. DATA QUALITY

 Missing values - TRAIN: 0, 

KeyboardInterrupt: 

2 - PREPROCESSING


In [ ]:
# Drop columns we don't need 

col_to_drop = ['cc_num','trans_num','first','last','street']

df_train = df_train.drop(columns=col_to_drop)
df_test = df_test.drop(columns=col_to_drop)

print(f" Dropped {len(col_to_drop)} columns")
print("test columns : \n \n",df_test.columns)
print("\n \n train columns : \n \n",df_train.columns)

KeyError: "['cc_num', 'trans_num', 'first', 'last', 'street'] not found in axis"

In [ ]:
# Add / Modify columns so that our model can understand the data 


# Convert to datetime type 
df_train['trans_date_trans_time'] = pd.to_datetime(df_train['trans_date_trans_time'])
df_test['trans_date_trans_time'] = pd.to_datetime(df_test['trans_date_trans_time'])


# Get hours (0 to 23)
df_train['hour'] = df_train['trans_date_trans_time'].dt.hour
df_test['hour'] = df_test['trans_date_trans_time'].dt.hour

# Get day of week (0 = Monday / 6= Sunday)
df_train['day_of_week'] = df_train['trans_date_trans_time'].dt.dayofweek
df_test['day_of_week'] = df_test['trans_date_trans_time'].dt.dayofweek

# Get day of month ( 1 to 31)
df_train['day_of_month'] = df_train['trans_date_trans_time'].dt.day
df_test['day_of_month'] = df_test['trans_date_trans_time'].dt.day

# Get month (1 to 12)
df_train['month'] = df_train['trans_date_trans_time'].dt.month
df_test['month'] = df_test['trans_date_trans_time'].dt.month

# Weekend or not ? 
df_train['is_weekend'] = (df_train['day_of_week'] >= 5).astype(int)
df_test['is_weekend'] = (df_test['day_of_week'] >= 5).astype(int)

# Function to categrize time of day
def get_time_period(hour):
    """ 
    Categorize hour into time periods
    0: Morning (low fraud risk)
    1: Afternoon (low fraud risk)
    2: Evening (moderate fraud risk)
    3: Night (high fraud risk)
    """
    if 5 <= hour < 12:
        return 0
    elif 12 <= hour < 18:
        return 1
    elif 18 <= hour < 23:
        return 2
    else:
        return 3

# Apply 
df_train['time_period'] = df_train['hour'].apply(get_time_period)
df_test['time_period'] = df_train['hour'].apply(get_time_period)



In [ ]:
# Drop columns we don't need anymore 
columns_to_drop = ['trans_date_trans_time']

df_train = df_train.drop(columns=columns_to_drop)
df_test = df_test.drop(columns=columns_to_drop)


# Analysis 
print("\n Fraud transaction by hour: ")
fraud_by_hour = df_train[df_train['is_fraud'] == 1].groupby('hour').size()
print(fraud_by_hour.sort_values(ascending=False).head(5))



 Fraud transaction by hour: 
hour
22    1931
23    1904
1      658
0      635
2      625
dtype: int64
